# Title
[]()

In [3]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [4]:
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
# from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict, create_text_dict_from_folder
from wrangling import filter_df_all_conditions
from summary_chain import *

In [ ]:

from response_processing import revive_chatbot, revive_chatbot_dict, save_instance_to_dict, process_chaining_results2

In [4]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

In [4]:
# from pypdf import PdfReader
import pandas as pd
import openai
import os
# from openai.embeddings_utils import get_embedding, cosine_similarity
import re

# Revive chatbot

## Load chatbot dict

In [12]:
filename = 'batch_Chaining_attributes_relevance_2023-05-02_1410.sav'
pickle_path = r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
chatbot_dict = loadpickle(filename, pickle_path)
chatbot_dict

Time completed: 2023-05-02 14:51:04.377476
Dictionary keys: ['text1_prompt00']


{'text1_prompt00': {'text': 'Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression\n\nDiscussion\nThe study aimed at investigating the effects of aging on muscle soreness and muscle damage following an intensive squat exercise protocol as well as on age-related effects of a recovery intervention in tightly controlled populations of young and master athletes regarding their absolute performance level. As an important experimental precondition, all markers of fatigue and recovery showed a significant within-subject effect of time of measurement with strikingly more pronounced changes from pre- to post-exercise (Figure 2), while CK peaked at post-24 in most individuals. Usually, the highest CK activity is found at 24 or 48h post-exercise. A mean CK increase of up to two times the baseline and up to 800U/l in some individuals can be attributed to structural damage to muscle cells 

In [15]:
chatbot_dict['text1_prompt00']['summaries_dict']

{'response_01': '"🏋️\u200d♀️ Muscle soreness and damage after intense exercise can affect athletes of all ages, according to a new study. 💪 Master athletes were found to recover just as quickly as their younger counterparts, contrary to previous beliefs. 🌊 However, a mixed-method recovery intervention (MMR) of cold water immersion and compression did not contribute to faster physical recovery, but did promote recovery of perceptual measures. 🤔 The effectiveness of MMR should be individualized and further studies are recommended. #fitness #exercise #recovery #athletes #MMR #science"',
 'response_02': 'Intensive squat exercises can lead to muscle soreness and damage, which affects recovery time. However, a study found that master athletes recover similarly to younger athletes despite aging. A mixed-method recovery intervention (MMR) did not significantly improve recovery of physical performance or muscle damage but did promote recovery of perceptual measures. It is recommended to individ

In [23]:
len(chatbot_dict['text1_prompt00']['simple_summary_dict'])
chatbot_dict['text1_prompt00']['simple_summary_dict']

{1: {'response_01': {0: {'simple summary choice': 1,
    'simplify task': 'Use language appropriate for',
    'audience': 'a lay audience',
    'full simplify task': 'Use language appropriate for for a lay audience',
    'simple summary': "A new study has found that intense exercise can make your muscles sore and cause damage, no matter how old you are. But the good news is that older athletes can recover just as quickly as younger ones, which is different from what we used to think. However, a treatment called mixed-method recovery (MMR), which involves using cold water and compression, didn't help people recover physically faster, but it did help them feel better mentally. It's important to remember that MMR might not work for everyone, and more research is needed. So if you're into fitness and exercise, keep this in mind!",
    'original summary': '"🏋️\u200d♀️ Muscle soreness and damage after intense exercise can affect athletes of all ages, according to a new study. 💪 Master athlet

## Update `revive_chatbot` function: Iteration 1.1

In [26]:
def revive_chatbot(chatbot):
    """
    Convert the dictionary of a chatbot attributes into a Chaining object.
    """
    new_chatbot = Chaining(chatbot['text'])
    print(f'Article title: {chatbot["article_title"]}')
    for key in chatbot.keys():
        setattr(new_chatbot, key, chatbot[key])
        print(f'\tNew chatbot attribute added: {key}')
        if type(getattr(new_chatbot, key)) == dict:
            print(f'\t\tAttribute dictionary keys: {[attr for attr in getattr(new_chatbot, key)]}')
    if not hasattr(chatbot[key], 'n_previous_prompts'):
        
        print(f'\tNew chatbot attribute added: n_previous_prompts')
        setattr(new_chatbot, 'n_previous_prompts',  dict())

    else:
        print(f'\tUpdating `n_previous_prompts`...')

    summaries_keys = [key for key in new_chatbot.summaries_dict.keys() if re.match(new_chatbot.response_regex, key)]
    print(f'\t\tPrevious number of prompts for summaries_dict: {len(summaries_keys)} {[key for key in summaries_keys]}')
    new_chatbot.n_previous_prompts['summaries'] = len(summaries_keys)
    new_chatbot.n_previous_prompts['simple_summary'] = len(new_chatbot.simple_summary_dict.keys())
    new_chatbot.n_previous_prompts['relevance'] = len(new_chatbot.relevance_dict.keys())

    print(f'\t\tPrevious number of prompts:')
    print(f'\t\t\tOriginal summaries: {len(summaries_keys)} {[key for key in summaries_keys]}')
    print(f'\t\t\tSimple summaries: {len(new_chatbot.simple_summary_dict.keys())}')
    print(f'\t\t\tAdded relevance: {len(new_chatbot.relevance_dict.keys())}')
    
    return new_chatbot


def revive_chatbot_dict(chatbot_dict, texts='all'):
    """
    Convert the dictionary of dictionaries of chatbot attributes into a dictionary of Chaining objects.

    Parameters:
        - chatbot_dict (dict): dictionary of dictionaries of chatbot attributes.
        - texts (list or 'all'): list of integers corresponding to the text prompts to revive. 
            If 'all', all text prompts will be revived.

    See "2023-05-01 test new prompts" notebook for example usage.
    """
    if texts == 'all':
        text_prompts_to_revive = [text for text in chatbot_dict.keys()]
    elif isinstance(texts, list):
        text_prompts_to_revive = [
            text_prompt for text_prompt in chatbot_dict.keys() for text in texts if f'{text}_' in text_prompt]
    else:
        raise TypeError("The `texts` parameter must be 'all' or a list of integers")

    new_chatbot_dict = {text_prompt: revive_chatbot(chatbot_dict[text_prompt]) for text_prompt in text_prompts_to_revive}
    print(f'\n\nNew chatbot dict keys: {[key for key in new_chatbot_dict]}')
    return new_chatbot_dict


new_chatbot_dict = revive_chatbot_dict(chatbot_dict, texts=[1])
new_chatbot_dict

Article title: Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compression
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribute dictionary keys: [1]
	New chatbot attribute added: relevance_dict
		Attribute dictionary keys: [1, 2]
	New

{'text1_prompt00': <summary_chain.Chaining at 0x11d7fb53250>}

In [27]:
new_chatbot_dict['text1_prompt00'].n_previous_prompts

{'summaries': 5, 'simple_summary': 1, 'relevance': 2}

In [28]:
new_chatbot_dict

{'text1_prompt00': <summary_chain.Chaining at 0x11d7fb53250>}

# See if save filename will have original time stamp 
Code copied from `process_chaining_results2` function

In [31]:
original_summary_time = next(iter(new_chatbot_dict.values())).date_created
description = f'prompt_chain_summaries_and_relevance_{original_summary_time}_revived'
description
# save_instance_to_dict(
#     new_chatbot_dict,
#     description=description,
#     save_json=False
# )

'prompt_chain_summaries_and_relevance_2023-05-02_1410_revived'

# Send new prompts

# Summarization

In [12]:
user_relevance_task

['Rewrite the summary to include an explanation of how it is relevant for']

In [17]:
relevance_prompts = user_relevance_task
relevance_dict = dict()
chaining_dict = dict()
chaining_dict[1] = new_chatbot_dict
pause_per_request = 0

iteration_id = 1
summary_iteration_id = iteration_id
relevance = prompt_chaining_dict(relevance_prompts, relevance_audience, relevance_dict, 
    chaining_dict[summary_iteration_id], iteration_id, prompt_column='relevance', 
    n_choices=1, pause_per_request=pause_per_request, summary_iteration_id=summary_iteration_id
    )

summary_iteration_id: 1
n_choices: 1
**text2_prompt00
relevance_iteration:  3
Task: Rewrite the summary to include an explanation of how it is relevant for seniors
summaries_keys: 
	 ['response_01', 'response_02', 'response_03', 'response_04', 'response_05']
		...Preparing to add relevance to response_01
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_02
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_03
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done sending request to GPT-3
	...Relevance statement given
		...Preparing to add relevance to response_04
	Done creating prompt
	Sending request to GPT-3
		Requesting 1 choices using gpt-3.5-turbo
	Done 

In [38]:
print(len(relevance_dict[1]))
relevance_dict[1]

4


[{1: {'response_01': {0: {'relevance choice': 1,
     'relevance task': 'Add 1-2 sentences to make this relevant for',
     'audience': 'seniors',
     'full relevance task': 'Add 1-2 sentences to make this relevant for seniors',
     'relevance statement': 'This study is particularly relevant for seniors who are interested in maintaining their physical fitness and preventing muscle soreness and damage. It suggests that master athletes, who are older athletes, do not experience higher levels of fatigue or slower recovery than younger athletes, indicating that age should not be a barrier to exercise. However, it is important to individualize recovery methods post-exercise to ensure optimal recovery.',
     'preceding summary': 'A recent study investigated the effects of aging on muscle soreness and damage following an intensive squat exercise protocol and the age-related effects of a recovery intervention in young and master athletes. The study found that master athletes do not experien

# *End of Page*